Fernando José Garavito Ovando 18071

Sebastián Maldonado Arnau 18003

# Laboratorio 2 Detección de SPAM

In [18]:
import pandas as pd
import numpy as np
import nltk, re, contractions
import unicodedata
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

## Parte 1 - Ingeniería de características

### Cargar datasets y unirlos

In [2]:
df1 = pd.read_csv("completeSpamAssassin.csv")
df1.head()

,Unnamed: 0,Body,Label
0,0,\nSave up to 70% on Life Insurance.\nWhy Spend...,1
1,1,1) Fight The Risk of Cancer!\nhttp://www.adcli...,1
2,2,1) Fight The Risk of Cancer!\nhttp://www.adcli...,1
3,3,##############################################...,1
4,4,I thought you might like these:\n1) Slim Down ...,1


In [3]:
df1 = df1.drop('Unnamed: 0', axis=1)
print(df1.shape)
df1.head()

(6046, 2)


,Body,Label
0,\nSave up to 70% on Life Insurance.\nWhy Spend...,1
1,1) Fight The Risk of Cancer!\nhttp://www.adcli...,1
2,1) Fight The Risk of Cancer!\nhttp://www.adcli...,1
3,##############################################...,1
4,I thought you might like these:\n1) Slim Down ...,1


In [4]:
df2 = pd.read_csv("enronSpamSubset.csv")
df2.head()

,Unnamed: 0.1,Unnamed: 0,Body,Label
0,2469,2469,Subject: stock promo mover : cwtd\n * * * urge...,1
1,5063,5063,Subject: are you listed in major search engine...,1
2,12564,12564,"Subject: important information thu , 30 jun 20...",1
3,2796,2796,Subject: = ? utf - 8 ? q ? bask your life with...,1
4,1468,1468,"Subject: "" bidstogo "" is places to go , things...",1


In [5]:
df2 = df2.drop('Unnamed: 0.1', axis=1)
df2 = df2.drop('Unnamed: 0', axis=1)
print(df2.shape)
df2.head()

(10000, 2)


,Body,Label
0,Subject: stock promo mover : cwtd\n * * * urge...,1
1,Subject: are you listed in major search engine...,1
2,"Subject: important information thu , 30 jun 20...",1
3,Subject: = ? utf - 8 ? q ? bask your life with...,1
4,"Subject: "" bidstogo "" is places to go , things...",1


In [6]:
df = pd.concat([df1,df2])
print(df.shape)
df.head()

(16046, 2)


,Body,Label
0,\nSave up to 70% on Life Insurance.\nWhy Spend...,1
1,1) Fight The Risk of Cancer!\nhttp://www.adcli...,1
2,1) Fight The Risk of Cancer!\nhttp://www.adcli...,1
3,##############################################...,1
4,I thought you might like these:\n1) Slim Down ...,1


### Preprocesamiento de texto

In [7]:
df['Body']

0       \nSave up to 70% on Life Insurance.\nWhy Spend...
1       1) Fight The Risk of Cancer!\nhttp://www.adcli...
2       1) Fight The Risk of Cancer!\nhttp://www.adcli...
3       ##############################################...
4       I thought you might like these:\n1) Slim Down ...
                              ...                        
9995    Subject: monday 22 nd oct\n louise ,\n do you ...
9996    Subject: missing bloomberg deals\n stephanie -...
9997    Subject: eops salary survey questionnaire\n we...
9998    Subject: q 3 comparison\n hi louise ,\n i have...
9999    Subject: confidential folder to safely pass in...
Name: Body, Length: 16046, dtype: object

In [8]:
stop_words = nltk.corpus.stopwords.words('english')
def procesar(doc):
    doc = str(doc)
    doc = re.sub(r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)','url',doc)
    doc = doc.lower()
    doc = contractions.fix(doc)
    doc = unicodedata.normalize('NFKD', doc).encode('ascii','ignore').decode('utf-8','ignore')
    doc = re.sub(r'[^a-z\s]','',doc)
    tokens = nltk.word_tokenize(doc)
    filtered = [token for token in tokens if token not in stop_words]
    filtered = filtered if len(filtered) < 50 else filtered[:49]
    doc = ' '.join(filtered)
    doc = re.sub(' +', ' ', doc)
    return doc
    

In [9]:
df['Body'] = df['Body'].apply(procesar)
df.sample(10)

,Body,Label
5541,found following mentioned newsgnomeorg usenet ...,0
6038,empty,0
994,public announcementthe new name domain extensi...,1
2510,subject med alert link complete pdf report cli...,1
1363,application grant remember type grant never ne...,1
15,want watch sporting eventsmoviespayperview ass...,1
6596,subject enron position understanding credit ar...,0
2867,subject actinium blackmail ouch email loading ...,1
3779,subject better economy akx finance even bad cr...,1
7308,subject enrononline weekly public report novem...,0


### Modelos BoW y TF-IDF

In [10]:
target = df['Label']
body = np.array(df['Body'])

#### BoW (n=1,2)

In [11]:
cv = CountVectorizer(min_df=0.01, ngram_range=(1,2))
cv_matrix = cv.fit_transform(body)
cv_matrix = cv_matrix.toarray()
vocabulario = cv.get_feature_names_out()
vocabulario.shape
bow = pd.DataFrame(cv_matrix, columns=vocabulario)
bow

,able,access,account,actually,add,additional,address,addresses,adobe,ago,...,would like,write,writes,wrote,www,xp,year,years,yes,yet
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16041,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16042,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16043,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16044,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### TF-IDF

In [12]:
tv = TfidfVectorizer(min_df=0.01, max_df=1., use_idf=True)
tv_matrix = tv.fit_transform(body)
tv_matrix = tv_matrix.toarray()
vocabulario = tv.get_feature_names_out()
tfidf = pd.DataFrame(np.round(tv_matrix, 2), columns=vocabulario)
tfidf

,able,access,account,actually,add,additional,address,addresses,adobe,ago,...,would,write,writes,wrote,www,xp,year,years,yes,yet
0,0.0,0.17,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
1,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
2,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
3,0.0,0.28,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.15
4,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16041,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
16042,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
16043,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
16044,0.0,0.00,0.0,0.0,0.26,0.0,0.0,0.00,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00


## Parte 2 - Implementación

### BoW

In [13]:
Bow_train, Bow_test, y_train, y_test = train_test_split(bow, target, test_size=0.3, random_state=104, shuffle=True)

In [17]:
clf = MultinomialNB()
clf = clf.fit(Bow_train,y_train)

In [19]:
y_pred = clf.predict(Bow_test)
print('Matriz de confusión:\n', metrics.confusion_matrix(y_test, y_pred))

Matriz de confusión:
 [[2419  341]
 [ 143 1911]]


In [20]:
print(metrics.classification_report(y_test, y_pred, target_names=['HAM', 'SPAM']))

              precision    recall  f1-score   support

         HAM       0.94      0.88      0.91      2760
        SPAM       0.85      0.93      0.89      2054

    accuracy                           0.90      4814
   macro avg       0.90      0.90      0.90      4814
weighted avg       0.90      0.90      0.90      4814



### TF-IDF

In [21]:
Tf_train, Tf_test, y_train, y_test = train_test_split(tfidf, target, test_size=0.3, random_state=104, shuffle=True)

In [22]:
clf2 = MultinomialNB()
clf2 = clf.fit(Tf_train,y_train)

In [23]:
y_pred = clf.predict(Tf_test)
print('Matriz de confusión:\n', metrics.confusion_matrix(y_test, y_pred))

Matriz de confusión:
 [[2542  218]
 [ 212 1842]]


In [24]:
print(metrics.classification_report(y_test, y_pred, target_names=['HAM', 'SPAM']))

              precision    recall  f1-score   support

         HAM       0.92      0.92      0.92      2760
        SPAM       0.89      0.90      0.90      2054

    accuracy                           0.91      4814
   macro avg       0.91      0.91      0.91      4814
weighted avg       0.91      0.91      0.91      4814



## Conclusión

El modelo de representación de texto que nos dió un mejor resultado fue el de TF-IDF ya que tiene un mayor f1-score y también mayor accuracy. Esto significa que es mejor prediciendo si el correo es spam o ham. Para ambos modelos el f1-score para ham es mayor que el de spam así que son mejores prediciendo qué correos son legítimos que los que son spam. Al comparar las matrices de confusión de ambos modelos podemos notar que el de fd-idf está más balanceado y tiene menores errores en los correos HAM aunque tiene un poco más de errores en los correos SPAM.  
  
En general ambos modelos quedaron muy similares y entregan resultados buenos (mayores a 90% de accuracy) pero es ligeramente mejor el modelo de tf-idf.